# Data source  

I found this [BarChart.com](https://www.barchart.com/etfs-funds/quotes/SPY/constituents?page=0) is providing spy constituents data. However, this website uses javascripts and getting html information is not straight forward. After some research, I decide to use a chrome webdrive to solve the issue.  

chrome WebDrive browser binary that incorporates python calls. Basically, this mimic a human browsing internet instead of a straight request call  

the drive could be downloaded at [google site](http://chromedriver.chromium.org/downloads)

In [214]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
import timestring
import json
import time

In [121]:
driver = webdriver.Chrome(executable_path='/home/wzh/local/chrome_driver/chromedriver')

In [122]:
url = 'https://www.barchart.com/etfs-funds/quotes/SPY/constituents?page={0}'

In [123]:
data = []
for i in range(7):
    if i == 0:
        date = ''.join(soup.find('span', {'class':'current-date'}).text.split(',')[-2:]).strip()
    
    driver.get(url.format(i))
    html = driver.execute_script('return document.documentElement.outerHTML')
    soup = BeautifulSoup(html)
    
    tickers = [tag.text.strip() for tag in soup.findAll('td', {'class': 'symbol text-left'})]
    weights = [tag.text.strip() for tag in soup.findAll('td', {'class': 'percent'})]
    shares = [tag.text.strip() for tag in soup.findAll('td', {'class': 'sharesHeld'})]
    
    if tickers:
        data.append(pd.DataFrame({
            'Ticker': tickers,
            'Weight': weights,
            'Share': shares
        }))


driver.quit()

/home/wzh/miniconda3/envs/edgar/lib/python3.6/site-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file /home/wzh/miniconda3/envs/edgar/lib/python3.6/runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))


# Do some clean up with data

In [180]:
df = pd.concat(data)

In [181]:
df = df.loc[~df.Ticker.isin(['Symbol', 'SPY', ''])]
df['Weight'] = df.Weight.str.strip('%').astype(float) / 100
df['Share'] = df.Share.str.replace( '[\$,)]','', regex=True ).astype(float)
asofdate = timestring.Date(date).date
df['AsOfDate'] = asofdate
print(asofdate)

2018-05-25 00:00:00


In [182]:
# the website gives duplicate entries when switching pages, need to groupby and dedup
df = df.groupby(['Ticker']).first().reset_index()

In [183]:
# change incorrect ticker 
df.loc[df.Ticker == 'WYND.I', 'Ticker'] = 'WYN'
df.loc[df.Ticker == 'CCL.U', 'Ticker'] = 'CCL'

In [184]:
df.to_csv(
    'constituents.{0}.csv'.format(asofdate.strftime('%Y%m%d')), 
    index=False)

# Go to wiki and find their company information

In [194]:
dfs = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies#cite_note-158')

In [195]:
company_info = dfs[0]
company_info.columns = company_info.iloc[0]
company_info = company_info.iloc[1:].rename(columns={'Ticker symbol':'Ticker'})
merged = company_info.merge(df, on='Ticker', how='outer', indicator=True)

In [196]:
merged.loc[merged._merge == 'left_only']

,Ticker,Security,SEC filings,GICS Sector,GICS Sub Industry,Location,Date first added[3][4],CIK,Founded,Weight,Share,AsOfDate,_merge


In [197]:
merged.loc[merged._merge == 'right_only']

,Ticker,Security,SEC filings,GICS Sector,GICS Sub Industry,Location,Date first added[3][4],CIK,Founded,Weight,Share,AsOfDate,_merge


In [198]:
merged.head()

,Ticker,Security,SEC filings,GICS Sector,GICS Sub Industry,Location,Date first added[3][4],CIK,Founded,Weight,Share,AsOfDate,_merge
0,MMM,3M Company,reports,Industrials,Industrial Conglomerates,"St. Paul, Minnesota",NaN,0000066740,1902,0.0051,6754508.0,2018-05-25,both
1,ABT,Abbott Laboratories,reports,Health Care,Health Care Equipment,"North Chicago, Illinois",1964-03-31,0000001800,1888,0.0046,19738462.0,2018-05-25,both
2,ABBV,AbbVie Inc.,reports,Health Care,Pharmaceuticals,"North Chicago, Illinois",2012-12-31,0001551152,2013 (1888),0.0072,18123128.0,2018-05-25,both
3,ACN,Accenture plc,reports,Information Technology,IT Consulting & Other Services,"Dublin, Ireland",2011-07-06,0001467373,1989,0.0041,7009123.0,2018-05-25,both
4,ATVI,Activision Blizzard,reports,Information Technology,Home Entertainment Software,"Santa Monica, California",2015-08-31,0000718877,2008,0.0023,8624631.0,2018-05-25,both


In [261]:
merged.loc[merged.Ticker == 'LUK', 'Ticker'] = 'LUK-W'
merged['Ticker'] = merged.Ticker.str.replace('.', '/')

In [262]:
merged.to_csv('company_info.{0}.csv'.format(asofdate.strftime('%Y%m%d')), index=False)

# Relate to Bloomberg to get more security information

In [263]:
tickers = merged.Ticker.values

In [264]:
headers = {'content-type': 'text/json'}
figi_url = 'https://api.openfigi.com/v1/mapping'
n = 10
res = []
for i in range(0, len(tickers), n):
    data = []
    group = tickers[i:i+n]
    for ticker in group:
        data.append({"idType":"TICKER",
                     "idValue":ticker.replace('.', '/'), 
                     "marketSecDes":"Equity", 
                     "exchCode":"US"})
        
    r = requests.post(figi_url, data=json.dumps(data), headers=headers)
    res = res + json.loads(r.content)
    time.sleep(20)
    

In [265]:
f = []
for d in res:
    try:
        f = f + d['data']
    except Exception as e:
        print(e)

In [266]:
bloomberg_figi = pd.DataFrame(f)

In [274]:
bloomberg_figi = bloomberg_figi.rename(columns={'compositeFIGI': 'composite_figi',
                               'exchCode': 'exch_code',
                               'figi': 'figi',
                               'marketSector': 'market_sector',
                               'name': 'name',
                               'securityDescription': 'security_description',
                               'securityType': 'security_type',
                               'securityType2': 'security_type2',
                               'shareClassFIGI': 'share_class_figi',
                               'ticker': 'ticker',
                               'uniqueID': 'unique_id',
                               'uniqueIDFutOpt': 'unique_id_futopt'})

In [277]:
bloomberg_figi.head()

,composite_figi,exch_code,figi,market_sector,name,security_description,security_type,security_type2,share_class_figi,ticker,unique_id,unique_id_futopt
0,BBG000BP52R2,US,BBG000BP52R2,Equity,3M CO,MMM,Common Stock,Common Stock,BBG001S5T7X2,MMM,EQ0010100100001000,None
1,BBG000B9ZXB4,US,BBG000B9ZXB4,Equity,ABBOTT LABORATORIES,ABT,Common Stock,Common Stock,BBG001S5N9M6,ABT,EQ0010000200001000,None
2,BBG0025Y4RY4,US,BBG0025Y4RY4,Equity,ABBVIE INC,ABBV,Common Stock,Common Stock,BBG0025Y4RZ3,ABBV,EQ0000000020553113,None
3,BBG000D9D830,US,BBG000D9D830,Equity,ACCENTURE PLC-CL A,ACN,Common Stock,Common Stock,BBG001SCXK90,ACN,EQ0030078700001000,None
4,BBG000CVWGS6,US,BBG000CVWGS6,Equity,ACTIVISION BLIZZARD INC,ATVI,Common Stock,Common Stock,BBG001S6C009,ATVI,EQ0010227300001000,None


In [276]:
bloomberg_figi.to_csv('bloomberg.csv', index=False)